In [1]:
using Mimi

In [2]:
include("helper_fund.jl")
include("helper_fairv2.jl")
include("helper_mesmer.jl")

load_region_lv (generic function with 1 method)

In [3]:
include("new_marginaldamages.jl")
include("components_fund/SocioEconomicComponent.jl")
include("components_fairv2/emissions.jl")
include("components_fairv2/co2_cycle.jl")
include("components_fairv2/ch4_cycle.jl")
include("components_fairv2/n2o_cycle.jl")
include("components_fairv2/montreal_gas_cycles.jl")
include("components_fairv2/flourinated_gas_cycles.jl")
include("components_fairv2/aerosol_plus_gas_cycles.jl")
include("components_fairv2/radiative_forcing.jl")
include("components_fairv2/temperature.jl")
include("components_fund/ClimateRegionalComponent.jl")
include("components_fund/ImpactAggregationComponent.jl")

In [4]:
function get_model(regions, esm::String, emu::Int,  emissions_forcing_scenario::String="ssp119", start_year::Int=1950, end_year::Int=2300)
    
    # ------------------------------------
    # Create model
    # ------------------------------------
    
    m = Model()
    
    # ------------------------------------
    # Set dimensions
    # ------------------------------------
    gas_p = DataFrame(load(joinpath(@__DIR__, "..", "data/fair_data", "default_gas_cycle_parameters.csv"), skiplines_begin=6))
    montreal_gas_p     = filter(:gas_group => ==("montreal"), gas_p)
    flourinated_gas_p  = filter(:gas_group => ==("flourinated"), gas_p)
    aerosol_plus_gas_p = filter(:gas_group => ==("aerosol_plus"), gas_p)
    
    set_dimension!(m, :time, start_year:end_year)
    set_dimension!(m, :regions, regions)
    set_dimension!(m, :montreal_gases, montreal_gas_p[:,:gas_name])
    set_dimension!(m, :flourinated_gases, flourinated_gas_p[:,:gas_name])
    set_dimension!(m, :aerosol_plus_gases, aerosol_plus_gas_p[:,:gas_name])
    
    # ------------------------------------
    # Create components
    # ------------------------------------

    add_comp!(m, socioeconomic)
    add_comp!(m, emissions)
    add_comp!(m, co2_cycle)
    add_comp!(m, ch4_cycle)
    add_comp!(m, n2o_cycle)
    add_comp!(m, montreal_cycles)
    add_comp!(m, flourinated_cycles)
    add_comp!(m, aerosol_plus_cycles)
    add_comp!(m, radiative_forcing)
    add_comp!(m, temperature)
    add_comp!(m, climateregional)
    add_comp!(m, impactaggregation)

    # ---------------------------------------------
    # Connect parameters to variables
    # ---------------------------------------------

    connect_param!(m, :climateregional, :inputtemp, :temperature, :T)    

    # ---------------------------------------------
    # Set all external parameter values
    # ---------------------------------------------
    update_MimiFAIR2_params!(m; emissions_forcing_scenario, start_year, end_year)
    
    region_lt_coef, region_lt_inter = load_region_lt(esm)
    region_lv = load_region_lv(esm,emu)
    
    #include backup data from 1990 to 2015 for emus_lv
    region_lv_backup = DataFrame(load(joinpath(@__DIR__, "..", "data","climateregional-emus_lv.csv")))
    region_lv = vcat(region_lv_backup,region_lv)
    
    mesmer_parameters = Dict{Tuple{Symbol,Symbol}, Any}()
    mesmer_parameters[(:climateregional, :region_lv)] = region_lv
    mesmer_parameters[(:climateregional, :region_lt_coef)] = region_lt_coef
    mesmer_parameters[(:climateregional, :region_lt_inter)] = region_lt_inter
    
    nsteps = length(start_year:end_year)
    
    prepparameters!(mesmer_parameters, nsteps)
    for (name, value) in mesmer_parameters
        update_param!(m, name[1], name[2], value)
    end
    
    update_fund_params!(m;nsteps)
    
    return m
end

get_model (generic function with 4 methods)

In [5]:
using DelimitedFiles
regionsdata = readdlm(joinpath(@__DIR__, "..", "data", "regions.csv"), ',', String)
regions =regionsdata[2:end, 1];

In [6]:
esm = "IPSL-CM6A-LR"
emu = 1
emissions_forcing_scenario="ssp119"
start_year = 1990
end_year = 2300

2300

In [7]:
m =  get_model(regions, esm, emu, emissions_forcing_scenario, start_year, end_year)

Mimi.Model
  Module: Mimi
  Components:
    ComponentId(Main.socioeconomic)
    ComponentId(Main.emissions)
    ComponentId(Main.co2_cycle)
    ComponentId(Main.ch4_cycle)
    ComponentId(Main.n2o_cycle)
    ComponentId(Main.montreal_cycles)
    ComponentId(Main.flourinated_cycles)
    ComponentId(Main.aerosol_plus_cycles)
    ComponentId(Main.radiative_forcing)
    ComponentId(Main.temperature)
    ComponentId(Main.climateregional)
    ComponentId(Main.impactaggregation)
  Built: false


In [8]:
run(m)

In [9]:
using CSV
gsc = compute_scco2(m, year = 2020,last_year=2300).gsc
show(gsc)
rsc = compute_scco2(m, year = 2020,last_year=2300).rsc
rsc = rsc[1,:]; # make a vector
df = DataFrame(
    :RegionID =>regions,
    :rsc => rsc,
);
#using CSV
#CSV.write("province_rsc.csv",df)
df.RegionID = parse.(Int, df.RegionID);

841.9727771103982

In [10]:
using PyCall,PyPlot
gpd = pyimport("geopandas")
pd = pyimport("pandas")

PyObject <module 'pandas' from 'C:\\ProgramData\\anaconda3\\envs\\claim_env\\lib\\site-packages\\pandas\\__init__.py'>

In [ ]:
countries = gpd.read_file(joinpath(@__DIR__, "..", "data/shapefile","subnational_boundaries.shp"))
df_py = pd.DataFrame(Dict([name => getproperty(df, name) for name in names(df)]))
merged_data = countries.merge(df_py, on="RegionID");
fig, ax = subplots()
ax[:set_title]("RSC Distribution")
merged_data.plot(column="rsc", ax=ax, legend=true, cmap="coolwarm")  # 使用colormap 'viridis'
show()
PyPlot.savefig("regional social cost of carbon.jpg", dpi=300)

In [ ]:
using VegaLite
using DataFrames

In [ ]:
temp = mean(m[:climateregional, :regtmp], dims = 2)[:,1];

df = DataFrame(
    :Year => Mimi.time_labels(m),
    :temp => temp)

stack(df, [:temp]) |> 

@vlplot(
    :line, 
    x = :Year,
    y = {:value, scale={domain=[18, 20]}},
    color = :variable
)

In [ ]:
loss = sum(m[:impactaggregation, :loss], dims = 2)[:,1];

df = DataFrame(
    :Year => Mimi.time_labels(m),
    :loss => loss)

stack(df, [:loss]) |> 

@vlplot(
    :line, 
    x = :Year,
    y = :value,
    color = :variable
)

In [ ]:
explore(m)

In [24]:
esms = [
    #"CAMS-CSM1-0",
    #"CanESM5",
    #"CanESM5-1",
    #"CNRM-ESM2-1",
    #"EC-Earth3-Veg-LR"
    #"GFDL-ESM4",
    #"GISS-E2-1-G",
    "IPSL-CM6A-LR",
    #"MIROC6",
    #"MPI-ESM1-2-LR",
    #"MRI-ESM2-0",
    #"UKESM1-0-LL",
]

1-element Vector{String}:
 "IPSL-CM6A-LR"

In [29]:
Regional_scc = DataFrame(:RegionID => regions)
for i in 1:2
    show("$i:")
    esm = rand(esms)
    emu = rand(1:1)
    m =  get_model(regions, esm, emu, emissions_forcing_scenario, start_year, end_year)
    run(m)
    gsc = compute_scco2(m, year = 2020,last_year=2300).gsc
    show(gsc)
    rsc = compute_scco2(m, year = 2020,last_year=2300).rsc
    rsc = rsc[1,:]; # make a vector
    emus = Symbol("emu_$i")
    df = DataFrame(
        :RegionID =>regions,
        emus => rsc,
    );
    Regional_scc =leftjoin(Regional_scc, df, on=:RegionID)
end
CSV.write("E:\\10Julia\\04research\\02SCC\\regional_scc.csv",Regional_scc)

"1:"865.6327438479061"2:"865.6327438479061

"E:\\10Julia\\04research\\02SCC\\regional_scc.csv"